In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings('ignore')

In [5]:
links = pd.read_csv("links.csv")
movies= pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")
tags = pd.read_csv("tags.csv")

In [6]:
links.head(1)

,movieId,imdbId,tmdbId
0,1,114709,862.0


In [7]:
movies.head(1)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [8]:
ratings.head(1)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703


In [9]:
tags.head(1)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994


In [12]:
print(movies.isnull().sum())
print(ratings.isnull().sum())
print(tags.isnull().sum())
print(links.isnull().sum())

movieId    0
title      0
genres     0
dtype: int64
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64
userId       0
movieId      0
tag          0
timestamp    0
dtype: int64
movieId    0
imdbId     0
tmdbId     8
dtype: int64


In [16]:
print(movies.duplicated().sum())
print(ratings.duplicated().sum())
print(tags.duplicated().sum())
print(links.duplicated().sum())

0
0
0
0


As this is most popular recommendation system, we only need movies and ratings. 
Later we can include all other info if required for the filtered movies. 

So, primarily working on the movies and ratings. 

In [19]:
ratings_with_names = ratings.merge(movies,on='movieId')

In [20]:
ratings_with_names.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [24]:
filter1 =ratings_with_names.groupby('title').count()['rating'].reset_index()
filter1.rename(columns={'rating':'Count'},inplace=True)
filter1

,title,Count
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2
...,...,...
9714,eXistenZ (1999),22
9715,xXx (2002),24
9716,xXx: State of the Union (2005),5
9717,¡Three Amigos! (1986),26


In [28]:
filter2 = ratings_with_names.groupby('title').mean()['rating'].reset_index()
filter2.rename(columns={'rating':'Average Rating'},inplace=True)
filter2

,title,Average Rating
0,'71 (2014),4.000000
1,'Hellboy': The Seeds of Creation (2004),4.000000
2,'Round Midnight (1986),3.500000
3,'Salem's Lot (2004),5.000000
4,'Til There Was You (1997),4.000000
...,...,...
9714,eXistenZ (1999),3.863636
9715,xXx (2002),2.770833
9716,xXx: State of the Union (2005),2.000000
9717,¡Three Amigos! (1986),3.134615


In [29]:
df1 = filter1.merge(filter2,on='title')
df1

,title,Count,Average Rating
0,'71 (2014),1,4.000000
1,'Hellboy': The Seeds of Creation (2004),1,4.000000
2,'Round Midnight (1986),2,3.500000
3,'Salem's Lot (2004),1,5.000000
4,'Til There Was You (1997),2,4.000000
...,...,...,...
9714,eXistenZ (1999),22,3.863636
9715,xXx (2002),24,2.770833
9716,xXx: State of the Union (2005),5,2.000000
9717,¡Three Amigos! (1986),26,3.134615


--- So we got the count and average of the movies. Now we put the filter for selecting the top 50 movies. 

In [32]:
df1 = df1[df1['Count'] >= 50].sort_values("Average Rating",ascending = False).head(50)
df1

,title,Count,Average Rating
7593,"Shawshank Redemption, The (1994)",317,4.429022
3499,"Godfather, The (1972)",192,4.289062
3011,Fight Club (1999),218,4.272936
1961,Cool Hand Luke (1967),57,4.271930
2531,Dr. Strangelove or: How I Learned to Stop Worr...,97,4.268041
6999,Rear Window (1954),84,4.261905
3500,"Godfather: Part II, The (1974)",129,4.259690
2334,"Departed, The (2006)",107,4.252336
3564,Goodfellas (1990),126,4.250000
1593,Casablanca (1942),100,4.240000


In [33]:
final_df = df1.merge(movies,on='title').drop_duplicates('title')[['movieId','title','genres']]
final_df

,movieId,title,genres
0,318,"Shawshank Redemption, The (1994)",Crime|Drama
1,858,"Godfather, The (1972)",Crime|Drama
2,2959,Fight Club (1999),Action|Crime|Drama|Thriller
3,1276,Cool Hand Luke (1967),Drama
4,750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War
5,904,Rear Window (1954),Mystery|Thriller
6,1221,"Godfather: Part II, The (1974)",Crime|Drama
7,48516,"Departed, The (2006)",Crime|Drama|Thriller
8,1213,Goodfellas (1990),Crime|Drama
9,912,Casablanca (1942),Drama|Romance


# DONE.